# Load and Index Data

In [266]:
from elasticsearch import Elasticsearch
import json

In [267]:
# Elastic search configuation
es = Elasticsearch(HOST="http://localhost", PORT=9200)

In [36]:
# Load file

file='harvard_dataverse.json'

with open(file, 'r', encoding='utf-8') as f:
    data=json.loads(f.read())

print(len(data))


100


In [37]:
for obj in data:
    #print("-----")
    obj_id=obj["@id"]
    es.index(index="harvardmetadata", doc_type="metadata", id=obj_id, body=obj)
    
    


In [39]:
es.indices.put_mapping(
    index="harvardmetadata",
    doc_type="metadata",
    include_type_name=True,
    body=
    {
        'properties': {
            '@context': {
                'type': 'text',
                'fields': {
                    'keyword': {
                        'type': 'keyword', 'ignore_above': 256}}},
    '@id': {'type': 'text',
     'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}},
    '@type': {'type': 'text',
     'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}},
    'author': {'properties': {'affiliation': {'type': 'text',
       'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}},
      'name': {'type': 'text',
       'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}}}},
    'citation': {'properties': {'@type': {'type': 'text',
       'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}},
      'text': {'type': 'text',
       'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}}}},
    'creator': {'properties': {'affiliation': {'type': 'text',
       'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}},
      'name': {'type': 'text',
       'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}}}},
    'dateModified': {'type': 'date'},
    'datePublished': {'type': 'date'},
    'description': {'type': 'text',
     'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}},
    'distribution': {'properties': {'@id': {'type': 'text',
       'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}},
      '@type': {'type': 'text',
       'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}},
      'contentSize': {'type': 'long'},
      'fileFormat': {'type': 'text',
       'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}},
      'identifier': {'type': 'text',
       'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}},
      'name': {'type': 'text',
       'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}}}},
    'funder': {'properties': {'@type': {'type': 'text',
       'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}},
      'name': {'type': 'text',
       'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}}}},
    'identifier': {'type': 'text',
     'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}},
    'includedInDataCatalog': {'properties': {'@type': {'type': 'text',
       'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}},
      'name': {'type': 'text',
       'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}},
      'url': {'type': 'text',
       'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}}}},
    'keywords': {'type': 'text',
     'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}},
    'license': {'properties': {'@type': {'type': 'text',
       'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}},
      'text': {'type': 'text',
       'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}}}},
    'name': {'type': 'text',
     'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}},
    'provider': {'properties': {'@type': {'type': 'text',
       'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}},
      'name': {'type': 'text',
       'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}}}},
    'publisher': {'properties': {'@type': {'type': 'text',
       'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}},
      'name': {'type': 'text',
       'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}}}},
    'version': {'type': 'text',
     'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}}}}
    
)


{'acknowledged': True}

## Testing Queries

In [268]:

query1={
        "query": {
            "match":{
              "@id": {
                "query": "https://doi.org/10.11588/data/0HJW2A"
              }
            }
        }
    }

In [269]:
q="https://doi.org/10.11588/data/10010"

In [276]:
field="keywords"
z= "Earth and Environmental Sciences"#'Epigraphik'#

In [277]:
id_query={
    "query": {
        "ids" : {
            "type" : "metadata",
            "values" : [q]
        }
    }
}


keyword_query={
  "query": {
    "match_phrase": {
        field: {
          "query": z
        }
    }
  }
}



In [278]:
res=es.search(index="harvardmetadata", doc_type="metadata", body=id_query)
res2=es.search(index="harvardmetadata", doc_type="metadata", body=keyword_query)


In [279]:
#print(res2)

hits=res2['hits']['total']['value']
if hits == 1:
    hit_obj = res['hits']['hits'][0]["_source"]
    data_json=json.dumps(hit_obj, separators=(',', ':'), indent=4)
    print(data_json)
if hits > 1:
    hits_list=res2["hits"]["hits"]
    data_list=[]
    for obj in hits_list:
        print("---")
        data_json=json.dumps(obj["_source"], separators=(',', ':'), indent=4)
        data_list.append(data_json)
        print(data_list)
        

---
['{\n    "@context":"http://schema.org",\n    "@type":"Dataset",\n    "@id":"https://doi.org/10.11588/data/3GYJUG",\n    "identifier":"https://doi.org/10.11588/data/3GYJUG",\n    "name":"MapSwipe: global dataset of crowdsourced classifications on human settlements and reference datasets",\n    "creator":[\n        {\n            "name":"Herfort, Benjamin",\n            "affiliation":"Institute of Geography, Heidelberg University"\n        }\n    ],\n    "author":[\n        {\n            "name":"Herfort, Benjamin",\n            "affiliation":"Institute of Geography, Heidelberg University"\n        }\n    ],\n    "datePublished":"2018-04-16",\n    "dateModified":"2018-04-16",\n    "version":"1",\n    "description":[\n        "MapSwipe is a mobile crowdsourcing application designed for smartphones and tablets to generate geographic information from satellite imagery. Data from 55 projects containing more than nine million tasks and more than 30 million individual classification resul

In [163]:
#es.indices.delete(index="testing")


In [280]:
mapping=es.indices.get_mapping(index="harvardmetadata")

In [281]:
mapping

{'harvardmetadata': {'mappings': {'properties': {'@context': {'type': 'text',
     'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}},
    '@id': {'type': 'text',
     'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}},
    '@type': {'type': 'text',
     'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}},
    'author': {'properties': {'@id': {'type': 'text',
       'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}},
      'affiliation': {'type': 'text',
       'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}},
      'identifier': {'type': 'text',
       'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}},
      'name': {'type': 'text',
       'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}}}},
    'citation': {'properties': {'@id': {'type': 'text',
       'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}},
      '@type': {'type': 'text',
       'fields': {'keyword': {'type'

In [ ]:
  'creator': {'properties': {'@id': {'type': 'text',
       'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}},
      'affiliation': {'type': 'text',
       'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}},
      'identifier': {'type': 'text',
       'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}},
      'name': {'type': 'text',
       'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}}}},

In [244]:
#mapping
fields=[]
for field in mapping["harvardmetadata"]['mappings']['properties']:
    #print(field)
    fields.append(field)
for field in mapping["harvardmetadata"]['mappings']['properties']:
    for x in mapping["harvardmetadata"]['mappings']['properties'][field]:
        if x == "properties":
            for leaf in mapping["harvardmetadata"]['mappings']['properties'][field]['properties']:
                leaf_id=field+"."+leaf
                fields.append(leaf_id)
                

In [245]:
print(fields)

['@context', '@id', '@type', 'author', 'citation', 'creator', 'dateModified', 'datePublished', 'description', 'distribution', 'funder', 'identifier', 'includedInDataCatalog', 'keywords', 'license', 'name', 'provider', 'publisher', 'spatialCoverage', 'temporalCoverage', 'version', 'author.@id', 'author.affiliation', 'author.identifier', 'author.name', 'citation.@id', 'citation.@type', 'citation.identifier', 'citation.text', 'creator.@id', 'creator.affiliation', 'creator.identifier', 'creator.name', 'distribution.@id', 'distribution.@type', 'distribution.contentSize', 'distribution.contentUrl', 'distribution.description', 'distribution.fileFormat', 'distribution.identifier', 'distribution.name', 'funder.@type', 'funder.name', 'includedInDataCatalog.@type', 'includedInDataCatalog.name', 'includedInDataCatalog.url', 'license.@type', 'license.text', 'license.url', 'provider.@type', 'provider.name', 'publisher.@type', 'publisher.name']


---